In [1]:
from tensorflow.keras.layers import (Dense,
                                    Dropout,
                                    Flatten,
                                    Input,
                                    concatenate,Embedding)
from tensorflow.keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import sys
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=sys.maxsize)

import warnings
warnings.filterwarnings(action='ignore')#, category=UserWarning, message=r"Boolean Series.*")

In [2]:
client1_df = pd.read_csv("./data/client1_train.csv",  sep=',', header=0,engine='python').fillna(0)
client2_df = pd.read_csv("./data/client2_train.csv",  sep=',', header=0,engine='python').fillna(0)
client3_df = pd.read_csv("./data/client3_train.csv",  sep=',', header=0,engine='python').fillna(0)
target_df = pd.read_csv("./data/train_credit_risk.csv",  sep=',', header=0,engine='python').fillna(0)

In [3]:
input_layers_1 = ['gsm_revn_l3m','tot_crdt_trxns_l6m','tot_send_mny_l1m']
input_layers_2 = ['max_paybill_l3m','mpsa_aon','days_bal_above50_l1m','recurrency_pay_l1m']
input_layers_3 = ['max_lnm_l12m','avg_dbit_amt_l1m','data_dou_l12m','rechg_days_l1m',
                  'tot_bor_airtm_trxns_l9m','tot_withdraw_trxns_l12m']

In [4]:
def prepare_train_data(client_df, target_df,input_layers):
    client_df = pd.merge(client_df, target_df, on='client_id',  how='left')
    X = client_df[input_layers]
    y = client_df.credit_risk
    dtypes = list(zip(X.dtypes.index, map(str, client_df.dtypes)))
    for k, dtype in dtypes:
        X[k] -= X[k].mean()
        X[k] /= X[k].std()
        X = X.fillna(0)
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=7)
    return X_train, X_valid, y_train, y_valid

In [5]:
X1_train, X1_valid, y1_train, y1_valid = prepare_train_data(client1_df, target_df,input_layers_1)
X2_train, X2_valid, y2_train, y2_valid = prepare_train_data(client2_df, target_df,input_layers_2)
X3_train, X3_valid, y3_train, y3_valid = prepare_train_data(client3_df, target_df,input_layers_3)

In [6]:
def build_model(X_train,y_train,X_valid,y_valid):
    input_els = []
    encoded_els = []
    dtypes = list(zip(X_train.dtypes.index, map(str, X_train.dtypes)))
    for k, dtype in dtypes:
        input_els.append(Input(name=f'{k}',shape=(1,)))
        if dtype == "int32":
            print(f'{k}: {X_train[k].max() + 1}')
            e = Flatten()(Embedding(X_train[k].max() + 3, 1,name=f'{k}_embeddings')(input_els[-1]))
        else:
            e = input_els[-1]
        encoded_els.append(e)
    encoded_els = concatenate(encoded_els,name='concatenate')
    layer0 = Dense(80, activation="relu",name='shared_0')(encoded_els)
    layer1 = Dense(75, activation="relu",name='shared_1')(layer0)
    layer2 = Dense(34, activation="relu",name='shared_2')(layer1)
    layer3 = Dense(23, activation="relu",name='shared_3')(layer2)
    layer4 = Dense(15, activation="relu",name='shared_4')(layer3)
    layer5 = Dense(5, activation="relu",name='shared_5')(layer4)
    layer6 = Dense(3, activation="relu",name='shared_6')(layer5)
    out    = Dense(1,name='output',activation="sigmoid")(layer4)

    # train model
    model = Model(inputs=input_els, outputs=[out])
    model.compile(optimizer="adam", loss="binary_crossentropy",metrics=[tf.keras.metrics.BinaryAccuracy()])
    model.fit(
        [X_train[k].values for k, t in dtypes],
        y_train,
        epochs=10,
        batch_size=16,
        shuffle=True,
        validation_data=([X_valid[k].values for k, t in dtypes], y_valid),
    )
    return model

In [7]:
def fed_weights_and_biases(model):
    weights = []
    biases = []
    num_layers = len(model.layers)
    
    for i in range(len(model.input)+2,num_layers):
        weight = model.layers[i].get_weights()[0]
        bias   = model.layers[i].get_weights()[1]

        weights.append(weight)
        biases.append(bias)
        
    return weights,biases

In [8]:
def average_weight(weights1,weights2,weights3,biases1,biases2,biases3):
    weight, bias=[],[]
    for i in range(len(weights2)):
        weight.append((weights1[i]+weights2[i]+weights3[i])/3)
        bias.append((biases1[i]+biases2[i]+biases3[i])/3)
    return weight, bias

In [9]:
def update_fed_weights_and_biases(model,weight,bias):
    num_layers = len(model.layers)
    
    for i in range(len(model.input)+2,num_layers):
        model.layers[i].set_weights([weight[i-num_layers],bias[i-num_layers]])
        
    return model

In [10]:
def federated_learning():
    
    epochs= 10
    model_1 = build_model(X1_train,y1_train,X1_valid,y1_valid)
    model_2 = build_model(X2_train,y2_train,X2_valid,y2_valid)
    model_3 = build_model(X3_train,y3_train,X3_valid,y3_valid)
    
    dtypes1 = list(zip(X1_train.dtypes.index, map(str, X1_train.dtypes)))
    dtypes2 = list(zip(X2_train.dtypes.index, map(str, X2_train.dtypes)))
    dtypes3 = list(zip(X3_train.dtypes.index, map(str, X3_train.dtypes)))
    print(f'START THE FEDERATED LEARNING PART')
    for main_loop in range(0,40):
        
        print(f'Federated Loop: {main_loop}')
        model_1.fit([X1_train[k].values for k, t in dtypes1],y1_train,epochs=epochs,batch_size=32,shuffle=True,
                            validation_data=([X1_valid[k].values for k, t in dtypes1], y1_valid))
        
        model_2.fit([X2_train[k].values for k, t in dtypes2],y2_train,epochs=epochs,batch_size=32,shuffle=True,
                            validation_data=([X2_valid[k].values for k, t in dtypes2], y2_valid))
        
        model_3.fit([X3_train[k].values for k, t in dtypes3],y3_train,epochs=epochs,batch_size=32,shuffle=True,
                            validation_data=([X3_valid[k].values for k, t in dtypes3], y3_valid))
        
        weights1 = model_1.layers[4].get_weights()[0]
        weights2 = model_2.layers[5].get_weights()[0]
        weights3 = model_3.layers[7].get_weights()[0]
        weights1,biases1 = fed_weights_and_biases(model_1)
        weights2,biases2 = fed_weights_and_biases(model_2)
        weights3,biases3 = fed_weights_and_biases(model_3)
        weight,bias = average_weight(weights1,weights2,weights3,biases1,biases2,biases3)
        model_1 = update_fed_weights_and_biases(model_1,weight,bias)
        model_2 = update_fed_weights_and_biases(model_2,weight,bias)
        model_3 = update_fed_weights_and_biases(model_3,weight,bias)
    return model_1, model_2, model_3

In [11]:
model_1, model_2, model_3 = federated_learning()

Epoch 1/10
2653/2653 [==============================] - 2s 744us/step - loss: 0.6090 - binary_accuracy: 0.6619 - val_loss: 0.6003 - val_binary_accuracy: 0.6721
Epoch 2/10
2653/2653 [==============================] - 2s 716us/step - loss: 0.6012 - binary_accuracy: 0.6683 - val_loss: 0.5969 - val_binary_accuracy: 0.6727
Epoch 3/10
2653/2653 [==============================] - 2s 722us/step - loss: 0.5988 - binary_accuracy: 0.6682 - val_loss: 0.5960 - val_binary_accuracy: 0.6702
Epoch 4/10
2653/2653 [==============================] - 2s 715us/step - loss: 0.5985 - binary_accuracy: 0.6698 - val_loss: 0.5981 - val_binary_accuracy: 0.6729
Epoch 5/10
2653/2653 [==============================] - 2s 716us/step - loss: 0.5978 - binary_accuracy: 0.6709 - val_loss: 0.5938 - val_binary_accuracy: 0.6710
Epoch 6/10
2653/2653 [==============================] - 2s 716us/step - loss: 0.5971 - binary_accuracy: 0.6715 - val_loss: 0.5938 - val_binary_accuracy: 0.6749
Epoch 7/10
2653/2653 [==================

Epoch 2/10
1327/1327 [==============================] - 1s 821us/step - loss: 0.5506 - binary_accuracy: 0.7096 - val_loss: 0.5556 - val_binary_accuracy: 0.7037
Epoch 3/10
1327/1327 [==============================] - 1s 812us/step - loss: 0.5506 - binary_accuracy: 0.7102 - val_loss: 0.5567 - val_binary_accuracy: 0.7057
Epoch 4/10
1327/1327 [==============================] - 1s 825us/step - loss: 0.5496 - binary_accuracy: 0.7114 - val_loss: 0.5573 - val_binary_accuracy: 0.7024
Epoch 5/10
1327/1327 [==============================] - 1s 824us/step - loss: 0.5497 - binary_accuracy: 0.7101 - val_loss: 0.5581 - val_binary_accuracy: 0.7025
Epoch 6/10
1327/1327 [==============================] - 1s 820us/step - loss: 0.5492 - binary_accuracy: 0.7097 - val_loss: 0.5570 - val_binary_accuracy: 0.7034
Epoch 7/10
1327/1327 [==============================] - 1s 835us/step - loss: 0.5490 - binary_accuracy: 0.7109 - val_loss: 0.5559 - val_binary_accuracy: 0.7050
Epoch 8/10
1327/1327 [==================

Epoch 3/10
1327/1327 [==============================] - 1s 793us/step - loss: 0.5431 - binary_accuracy: 0.7191 - val_loss: 0.5458 - val_binary_accuracy: 0.7181
Epoch 4/10
1327/1327 [==============================] - 1s 809us/step - loss: 0.5429 - binary_accuracy: 0.7180 - val_loss: 0.5459 - val_binary_accuracy: 0.7134
Epoch 5/10
1327/1327 [==============================] - 1s 813us/step - loss: 0.5423 - binary_accuracy: 0.7185 - val_loss: 0.5460 - val_binary_accuracy: 0.7166
Epoch 6/10
1327/1327 [==============================] - 1s 849us/step - loss: 0.5424 - binary_accuracy: 0.7187 - val_loss: 0.5468 - val_binary_accuracy: 0.7180
Epoch 7/10
1327/1327 [==============================] - 1s 819us/step - loss: 0.5422 - binary_accuracy: 0.7193 - val_loss: 0.5476 - val_binary_accuracy: 0.7194
Epoch 8/10
1327/1327 [==============================] - 1s 790us/step - loss: 0.5418 - binary_accuracy: 0.7192 - val_loss: 0.5440 - val_binary_accuracy: 0.7163
Epoch 9/10
1327/1327 [==================

Epoch 4/10
1327/1327 [==============================] - 1s 778us/step - loss: 0.5938 - binary_accuracy: 0.6741 - val_loss: 0.5922 - val_binary_accuracy: 0.6748
Epoch 5/10
1327/1327 [==============================] - 1s 743us/step - loss: 0.5932 - binary_accuracy: 0.6733 - val_loss: 0.5957 - val_binary_accuracy: 0.6744
Epoch 6/10
1327/1327 [==============================] - 1s 742us/step - loss: 0.5936 - binary_accuracy: 0.6757 - val_loss: 0.5928 - val_binary_accuracy: 0.6731
Epoch 7/10
1327/1327 [==============================] - 1s 741us/step - loss: 0.5936 - binary_accuracy: 0.6739 - val_loss: 0.5928 - val_binary_accuracy: 0.6750
Epoch 8/10
1327/1327 [==============================] - 1s 757us/step - loss: 0.5934 - binary_accuracy: 0.6738 - val_loss: 0.5938 - val_binary_accuracy: 0.6739
Epoch 9/10
1327/1327 [==============================] - 1s 764us/step - loss: 0.5934 - binary_accuracy: 0.6743 - val_loss: 0.5929 - val_binary_accuracy: 0.6739
Epoch 10/10
1327/1327 [=================

1327/1327 [==============================] - 1s 822us/step - loss: 0.5415 - binary_accuracy: 0.7161 - val_loss: 0.5610 - val_binary_accuracy: 0.6973
Epoch 6/10
1327/1327 [==============================] - 1s 812us/step - loss: 0.5410 - binary_accuracy: 0.7177 - val_loss: 0.5603 - val_binary_accuracy: 0.7037
Epoch 7/10
1327/1327 [==============================] - 1s 809us/step - loss: 0.5409 - binary_accuracy: 0.7163 - val_loss: 0.5609 - val_binary_accuracy: 0.6999
Epoch 8/10
1327/1327 [==============================] - 1s 816us/step - loss: 0.5403 - binary_accuracy: 0.7176 - val_loss: 0.5602 - val_binary_accuracy: 0.7021
Epoch 9/10
1327/1327 [==============================] - 1s 808us/step - loss: 0.5394 - binary_accuracy: 0.7183 - val_loss: 0.5617 - val_binary_accuracy: 0.6996
Epoch 10/10
1327/1327 [==============================] - 1s 810us/step - loss: 0.5390 - binary_accuracy: 0.7172 - val_loss: 0.5616 - val_binary_accuracy: 0.6995
Federated Loop: 6
Epoch 1/10
1327/1327 [==========

...

1327/1327 [==============================] - 1s 752us/step - loss: 0.5925 - binary_accuracy: 0.6755 - val_loss: 0.5936 - val_binary_accuracy: 0.6747
Epoch 2/10
1327/1327 [==============================] - 1s 739us/step - loss: 0.5916 - binary_accuracy: 0.6762 - val_loss: 0.5941 - val_binary_accuracy: 0.6742
Epoch 3/10
1327/1327 [==============================] - 1s 742us/step - loss: 0.5916 - binary_accuracy: 0.6758 - val_loss: 0.5942 - val_binary_accuracy: 0.6732
Epoch 4/10
1327/1327 [==============================] - 1s 738us/step - loss: 0.5914 - binary_accuracy: 0.6764 - val_loss: 0.5933 - val_binary_accuracy: 0.6743
Epoch 5/10
1327/1327 [==============================] - 1s 804us/step - loss: 0.5914 - binary_accuracy: 0.6759 - val_loss: 0.5934 - val_binary_accuracy: 0.6727
Epoch 6/10
1327/1327 [==============================] - 1s 736us/step - loss: 0.5912 - binary_accuracy: 0.6779 - val_loss: 0.5934 - val_binary_accuracy: 0.6738
Epoch 7/10
1327/1327 [=============================

1327/1327 [==============================] - 1s 809us/step - loss: 0.4951 - binary_accuracy: 0.7480 - val_loss: 0.6302 - val_binary_accuracy: 0.6872
Epoch 3/10
1327/1327 [==============================] - 1s 807us/step - loss: 0.4924 - binary_accuracy: 0.7487 - val_loss: 0.6416 - val_binary_accuracy: 0.6804
Epoch 4/10
1327/1327 [==============================] - 1s 844us/step - loss: 0.4875 - binary_accuracy: 0.7525 - val_loss: 0.6504 - val_binary_accuracy: 0.6798
Epoch 5/10
1327/1327 [==============================] - 1s 805us/step - loss: 0.4868 - binary_accuracy: 0.7542 - val_loss: 0.6533 - val_binary_accuracy: 0.6763
Epoch 6/10
1327/1327 [==============================] - 1s 832us/step - loss: 0.4852 - binary_accuracy: 0.7524 - val_loss: 0.6592 - val_binary_accuracy: 0.6791
Epoch 7/10
1327/1327 [==============================] - 1s 808us/step - loss: 0.4844 - binary_accuracy: 0.7543 - val_loss: 0.6589 - val_binary_accuracy: 0.6797
Epoch 8/10
1327/1327 [=============================

1327/1327 [==============================] - 1s 788us/step - loss: 0.5228 - binary_accuracy: 0.7318 - val_loss: 0.5731 - val_binary_accuracy: 0.7112
Epoch 4/10
1327/1327 [==============================] - 1s 789us/step - loss: 0.5226 - binary_accuracy: 0.7317 - val_loss: 0.5764 - val_binary_accuracy: 0.7121
Epoch 5/10
1327/1327 [==============================] - 1s 792us/step - loss: 0.5222 - binary_accuracy: 0.7326 - val_loss: 0.5771 - val_binary_accuracy: 0.7129
Epoch 6/10
1327/1327 [==============================] - 1s 791us/step - loss: 0.5212 - binary_accuracy: 0.7335 - val_loss: 0.5796 - val_binary_accuracy: 0.7129
Epoch 7/10
1327/1327 [==============================] - 1s 790us/step - loss: 0.5213 - binary_accuracy: 0.7333 - val_loss: 0.5748 - val_binary_accuracy: 0.7116
Epoch 8/10
1327/1327 [==============================] - 1s 791us/step - loss: 0.5209 - binary_accuracy: 0.7332 - val_loss: 0.5852 - val_binary_accuracy: 0.7085
Epoch 9/10
1327/1327 [=============================

1327/1327 [==============================] - 1s 744us/step - loss: 0.5908 - binary_accuracy: 0.6765 - val_loss: 0.5984 - val_binary_accuracy: 0.6745
Epoch 5/10
1327/1327 [==============================] - 1s 741us/step - loss: 0.5915 - binary_accuracy: 0.6764 - val_loss: 0.5978 - val_binary_accuracy: 0.6743
Epoch 6/10
1327/1327 [==============================] - 1s 744us/step - loss: 0.5908 - binary_accuracy: 0.6758 - val_loss: 0.6012 - val_binary_accuracy: 0.6733
Epoch 7/10
1327/1327 [==============================] - 1s 803us/step - loss: 0.5908 - binary_accuracy: 0.6766 - val_loss: 0.5943 - val_binary_accuracy: 0.6728
Epoch 8/10
1327/1327 [==============================] - 1s 743us/step - loss: 0.5907 - binary_accuracy: 0.6771 - val_loss: 0.5992 - val_binary_accuracy: 0.6735
Epoch 9/10
1327/1327 [==============================] - 1s 746us/step - loss: 0.5908 - binary_accuracy: 0.6762 - val_loss: 0.6087 - val_binary_accuracy: 0.6742
Epoch 10/10
1327/1327 [============================

1327/1327 [==============================] - 1s 821us/step - loss: 0.4815 - binary_accuracy: 0.7535 - val_loss: 0.6755 - val_binary_accuracy: 0.6769
Epoch 6/10
1327/1327 [==============================] - 1s 823us/step - loss: 0.4790 - binary_accuracy: 0.7582 - val_loss: 0.6653 - val_binary_accuracy: 0.6741
Epoch 7/10
1327/1327 [==============================] - 1s 830us/step - loss: 0.4780 - binary_accuracy: 0.7573 - val_loss: 0.6773 - val_binary_accuracy: 0.6765
Epoch 8/10
1327/1327 [==============================] - 1s 811us/step - loss: 0.4773 - binary_accuracy: 0.7578 - val_loss: 0.6660 - val_binary_accuracy: 0.6782
Epoch 9/10
1327/1327 [==============================] - 1s 853us/step - loss: 0.4769 - binary_accuracy: 0.7580 - val_loss: 0.6768 - val_binary_accuracy: 0.6795
Epoch 10/10
1327/1327 [==============================] - 1s 921us/step - loss: 0.4751 - binary_accuracy: 0.7584 - val_loss: 0.6996 - val_binary_accuracy: 0.6726


In [12]:
model_1.save(f'./model/client_1')
model_2.save(f'./model/client_2')
model_3.save(f'./model/client_3')

INFO:tensorflow:Assets written to: ./model/client_1/assets


INFO:tensorflow:Assets written to: ./model/client_1/assets


INFO:tensorflow:Assets written to: ./model/client_2/assets


INFO:tensorflow:Assets written to: ./model/client_2/assets


INFO:tensorflow:Assets written to: ./model/client_3/assets


INFO:tensorflow:Assets written to: ./model/client_3/assets


# Scoring the Federal Model

In [13]:
def prepare_score_data(client_df,input_layers):
    X = client_df[input_layers]
    dtypes = list(zip(X.dtypes.index, map(str, client_df.dtypes)))
    for k, dtype in dtypes:
        X[k] -= X[k].mean()
        X[k] /= X[k].std()
        X = X.fillna(0)
    score_data = []
    for i in range(X.shape[1]):
        score_data.append(X.iloc[:,i])
    return score_data

In [14]:
pred_client1_df = pd.read_csv("./data/client1_pred.csv",  sep=',', header=0,engine='python').fillna(0)
pred_client2_df = pd.read_csv("./data/client2_pred.csv",  sep=',', header=0,engine='python').fillna(0)
pred_client3_df = pd.read_csv("./data/client3_pred.csv",  sep=',', header=0,engine='python').fillna(0)
# target_df = pd.read_csv("./data/train_credit_risk.csv",  sep=',', header=0,engine='python').fillna(0)

In [15]:
input_layers_1 = ['gsm_revn_l3m','tot_crdt_trxns_l6m','tot_send_mny_l1m']
input_layers_2 = ['max_paybill_l3m','mpsa_aon','days_bal_above50_l1m','recurrency_pay_l1m']
input_layers_3 = ['max_lnm_l12m','avg_dbit_amt_l1m','data_dou_l12m','rechg_days_l1m',
                  'tot_bor_airtm_trxns_l9m','tot_withdraw_trxns_l12m']

In [16]:
pred_client1_df.shape

(2002, 4)

In [17]:
score1= prepare_score_data(pred_client1_df,input_layers_1)
score2= prepare_score_data(pred_client2_df,input_layers_2)
score3= prepare_score_data(pred_client3_df,input_layers_3)

In [18]:
y_client_1=model_1.predict(score1)
y_client_2=model_2.predict(score2)
y_client_3=model_3.predict(score3)

63/63 [==============================] - 0s 459us/step


In [19]:
# y_client_1[y_client_1>0.5]
# y_client_1

In [20]:
# y_client_2#[y_client_2>0.5]

In [21]:
# y_client_3[y_client_3>0.5]

In [22]:
np.sum((y_client_1>0.5)+(y_client_2>0.5)+(y_client_3>0.5))

1303

In [23]:
np.sum((y_client_1>0.5)|(y_client_2>0.5)|(y_client_3>0.5))

1303

In [24]:
np.sum((y_client_1>0.5)&(y_client_2>0.5)&(y_client_3>0.5))

327

In [25]:
pred_client1_df.shape

(2002, 4)

In [26]:
pred_client1_df.head()

client_id  gsm_revn_l3m  tot_crdt_trxns_l6m  tot_send_mny_l1m
0     839890      712.9539                18.0             765.0
1     755993     2256.9984               355.0            2520.0
2     880547      226.9357                68.0            5346.0
3     684412        0.0000                83.0            5680.0
4     740152     6530.8640               238.0           23802.0

In [27]:
pred_client1_df['prediction'] = np.round(y_client_1)
pred_client2_df['prediction'] = np.round(y_client_2)
pred_client3_df['prediction'] = np.round(y_client_3)

In [28]:
pred_client1_df.prediction.sum()

825.0

In [29]:
pred_client2_df.prediction.sum()

866.0

In [30]:
pred_client3_df.prediction.sum()

686.0

In [31]:
# pred_client1_df.prediction+pred_client2_df.prediction

In [32]:
preds = (pred_client1_df.prediction+pred_client2_df.prediction+pred_client3_df.prediction)>0

In [33]:
pred_client1_df['prediction'] = 1*preds

In [34]:
# pred_client1_df

In [35]:
evaluation_df = pred_client1_df[['client_id','prediction']]

In [36]:
evaluation_df.to_csv('group_4_v3.csv', index=False) 

In [37]:
evaluation_df.head()

client_id  prediction
0     839890           1
1     755993           0
2     880547           1
3     684412           0
4     740152           0